In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
from textblob import TextBlob

In [6]:
# %%
os.chdir("..")  # Go up a directory
#print(os.getcwd())

# %%
print(os.getcwd())

c:\Users\Belay\News-Sentiment-Analysis-Stock


In [ ]:

def prepare_stock_prediction_data(file_paths: dict, start_date, end_date, target_ticker, target_column: str = 'Close') -> pd.DataFrame:

    data_frames = []

    for ticker, path in file_paths.items():
        if not os.path.isfile(path):
            raise FileNotFoundError(f"{ticker} CSV not found at {path}")
        
        try:
            df = pd.read_csv(path, parse_dates=['Date'])
        except Exception as e:
            raise RuntimeError(f"Failed to read {ticker} CSV: {e}")
        
        if 'Date' not in df.columns or target_column not in df.columns:
            raise ValueError(f"{ticker} file must contain 'Date' and '{target_column}' columns")

        df = df[['Date', target_column]].copy()
        df.rename(columns={target_column: f"{ticker}_{target_column}"}, inplace=True)
        data_frames.append(df)

    try:
        merged_df = data_frames[0]
        for df in data_frames[1:]:
            merged_df = pd.merge(merged_df, df, on='Date', how='inner')
    except Exception as e:
        raise RuntimeError(f"Error merging DataFrames: {e}")

    # Filter by date range
    try:
        mask = (merged_df['Date'] >= pd.to_datetime(start_date)) & (merged_df['Date'] <= pd.to_datetime(end_date))
        merged_df = merged_df.loc[mask].copy()
        merged_df.sort_values('Date', inplace=True)
        merged_df.reset_index(drop=True, inplace=True)
    except Exception as e:
        raise ValueError(f"Date filtering failed: {e}")

    # Drop missing values
    merged_df.dropna(inplace=True)

    # Create target column: next-day close price for target_ticker
    try:
        merged_df[f"{target_ticker}_target"] = merged_df[f"{target_ticker}_{target_column}"].shift(-1)
        merged_df.dropna(inplace=True)
    except Exception as e:
        raise RuntimeError(f"Failed to create target column: {e}")

    return merged_df


In [20]:
file_paths = {
    'AAPL': 'Data/AAPL_historical_data.csv',
    'MSFT': 'Data/MSFT_historical_data.csv',
    'AMZN': 'Data/AMZN_historical_data.csv',
    'GOOG': 'Data/GOOG_historical_data.csv',
    'TSLA': 'Data/TSLA_historical_data.csv',
    'NVDA': 'Data/NVDA_historical_data.csv',
    'META': 'Data/META_historical_data.csv'
}

df = prepare_stock_data(file_paths, '1980-01-01', '2024-07-30','AAPL')

# View the first few rows
print(df.head())


TypeError: prepare_stock_data() takes from 1 to 3 positional arguments but 4 were given